# Calculation-of-employee-turnover-costs
 

## Project kickoff

### Importing libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn as skl
%matplotlib inline

ModuleNotFoundError: No module named 'pandas'

### DATA UPLOAD

Before continuing you must:

* Have downloaded the data from the platform

* Have unzipped them in a folder on your hard drive

* Have uploaded them to Colab using the file explorer (folder icon - upload file icon)

In [2]:
df = pd.read_csv("/kaggle/input/employees-turnover-costs-calculations/Employee turnover costs calculations.csv", sep=",", index_col="id", na_values="#N/D")
df

NameError: name 'pd' is not defined

## BUSINESS ANALYTICS

In [ ]:
df.info()

NameError: name 'df' is not defined

In [ ]:
df.isna().sum().sort_values(ascending = False)

### Conclusions
* Years in position and reconciliation have too many nulls --> eliminate variables
* Sex, education, job_satisfaction and involvement --> impute them after EDA

In [ ]:
df.drop(columns= ['years_in_position','conciliation'], inplace= True)


In [ ]:
df  

### EDA Categorical Variables

In [ ]:
def graficos_eda_categoricos(cat):
    #Calculamos el numero de filas que necesitamos
    from math import ceil
    filas = ceil(cat.shape[1]/2)
    
    #Definimos el grafico
    f,ax = plt.subplots(nrows = filas, ncols = 2, figsize = (16,filas * 6))
    
    #Aplanamos para iterar por el grafico como si fuera de 1 dimenson en lugar de 2
    ax = ax.flat
    
    #Creamos el bucle que va anadiendo graficos
    for cada, variable in enumerate(cat):
        cat [variable].value_counts().plot.barh(ax = ax[cada])
        ax[cada].set_title(variable, fontsize = 12,fontweight = "bold")
        ax[cada].tick_params(labelsize = 12)
        
    
    

In [ ]:
graficos_eda_categoricos(df.select_dtypes('O'))

### Cloncusiones
* mayor edad solo tiene un valor --> elimarla
* sobre las imputaciones pendientes de variable categoricas:
    * educacion: imputar por "Universitaria"
    * satisfacion trabajo: imputar por "Alta"
    * implicacion: imputar por "ALta"

In [ ]:
df.drop(columns = 'mayor_edad', inplace = True)
df['educacion'] = df['educacion'].fillna('Universitaria')
df['satisfaccion_trabajo'] = df['satisfaccion_trabajo'].fillna('Alta')
df['implicacion'] = df['implicacion'].fillna('Alta')

In [ ]:
df.info()

### EDA VARIABLES NUMERICAS


In [ ]:
def estadisticos_cont(num):
    #calculamos describe
    estadisticos = num.describe().T
    #anadimos la mediana
    estadisticos['median'] = num.median()
    #reordenamos pra que la mediana este al lado de la media
    estadisticos= estadisticos.iloc[:,[0,1,8,2,3,4,5,6,7]]
    #lo devolvemos
    return(estadisticos)

In [ ]:
estadisticos_cont(df.select_dtypes('number'))

### Conclusiones
* Empleados solo tiene un valor --> elimarlo
* sexo tiene 3 valores --> eliminarlo
* horas quincena solo tiene un valor --> eliminarlo

In [ ]:
df.drop(columns=['empleados','sexo','horas_quincena'], inplace= True)


In [ ]:
df

### GENERACION DE INSIGHTS

Cuantification del problema: Cual es la tasa de abandono?

In [ ]:
df.abandono.value_counts(normalize = True)* 100

Hay un perfil tipo de empleado que deja la empresa?

In [ ]:
#Transformar abondono a numerico
df['abandono'] = df['abandono'].replace({'No': 0, 'Yes': 1})

# Imprimir el DataFrame
df

In [ ]:
#Analisis por educacion
temp = df.groupby('educacion').abandono.mean().sort_values(ascending = False)*1000
temp.plot.bar();

In [ ]:
# Analisis por esta do civil

In [ ]:
temp = df.groupby('estado_civil').abandono.mean().sort_values(ascending = False)*1000
temp.plot.bar();

In [ ]:
temp = df.groupby('puesto').abandono.mean().sort_values(ascending = False)*1000
temp.plot.bar();

In [ ]:
temp = df.groupby('abandono').salario_mes.mean()
temp.plot.bar();

### Cloncusiones
El perfil medio del empleado que deja la empresa es:
 * bajo nivel de estudios
 * soltero
 * trabaja en ventas
 * Bajo salario
* alta carga de horas extras
    

Cual es el impacto economico de este proble,ma?

Segun el estudio "Cot of turnover"

el costo de la fuga de los empleados que ganan menos de 30000 es del 16.1% de su salario.
el costo de la fuga de los empleados que ganan entre  30000 y 50000 es del 19.7% de su salario.
el costo de la fuga de los empleados que ganan entre  50000 y 75000 es del 20.4% de su salario.
el costo de la fuga de los empleados que ganan mas de 75000 es del 21% de su salario.

In [ ]:
#Creamos una nueva variable salario_ano del empleado
df['salario_ano'] =df.salario_mes.transform(lambda x: x*12)
df[['salario_mes','salario_ano']]

In [ ]:
# Calcular el impacto económico
condiciones = [(df['salario_ano'] <= 30000),
               (df['salario_ano'] > 30000) & (df['salario_ano']<= 50000),
               (df['salario_ano'] > 50000) & (df['salario_ano']<= 75000),
               (df['salario_ano'] > 75000)]

resultados = [df.salario_ano *0.161, df.salario_ano * 0.197, df.salario_ano * 0.204, df.salario_ano * 0.21]

df['impacto_abandono'] = np.select(condiciones,resultados,default= -999)

# Imprimir el DataFrame
df


Cuanto nos ha costado este problema el ultimo ano?

In [ ]:
coste_total = df.loc[df.abandono == 1].impacto_abandono.sum()
coste_total

Cuanto nos custa que los empleados no esten motivados? perdidas en implicacion == baja

In [ ]:
df.loc[(df.abandono == 1)& (df.implicacion == 'Baja')].impacto_abandono.sum()

Cuanto dinero podriamos ahorrar fidellizando mejor a nuestro empleados?

In [ ]:
print(f"Reducir un 10% la fuga de empleados nos ahorraria {int(coste_total *  0.1)} $ cada ano")
print(f"Reducir un 20% la fuga de empleados nos ahorraria {int(coste_total *  0.2)} $ cada ano")
print(f"Reducir un 20% la fuga de empleados nos ahorraria {int(coste_total *  0.2)} $ cada ano")

#### Y podemos seguir tranzando estrategias asociadas a los insights de abondono:

Habiamos visto que los reprensantes de ventas son el puesto que mas se va. Tendria sentido hacer un plan especifico para ellos? Cual seria el coste ahorrado si disminuimos la fuga un 30%?
Primero vamos a calcular el % de reprensentantes de ventas que se ahan ido el ano pasado.

In [ ]:
total_repre_pasado = len(df.loc[df.puesto == 'Sales Representative'])
abandonos_repre_pasado = len(df.loc[(df.puesto == 'Sales Representative') & (df.abandono == 1)])
porc_pasado = abandonos_repre_pasado / total_repre_pasado

porc_pasado

Ahora vamos a cuantificar cuantos se nos iran este ano

In [ ]:
total_repre_actual = len(df.loc[(df.puesto == 'Sales Representative') & (df.abandono == 0)])
se_iran = int(total_repre_actual * porc_pasado)
se_iran

Sobre ellos cuantos podemos retener (hipotesis 30%) y cuanto dinero puee suponer

In [ ]:
retenemos = int(se_iran * 0.3)
ahorramos = df.loc[(df.puesto == 'Sales Representative') & (df.abandono == 0), 'impacto_abandono'].sum()*porc_pasado*0.3
print(f'Podemos retener {retenemos} representantes de ventas y ello supondria ahorrar {ahorramos}')

Este dato tambien es muy interesante porque nos permite determinar el presupuesto para acciones de retencion por departamento o perfil.
Ya que sabemos que podemos gastarnos hasta 37.000$ solo en acciones especificas para retener a representantes de ventas y se estarian solo pagando con la perdida evitada.

## DIA 3 : MODELO DE MACHINE LEARNING

In [ ]:
df_ml= df.copy()
 
df_ml.info()

### PREPARACION DE LOS DATOS PARA LA MODELIZAION

#### Transformar todas las variables categoricas a numericas

In [ ]:
from sklearn.preprocessing import OneHotEncoder

#Categoricas
cat = df_ml.select_dtypes('O')

#Instalaciones

ohe = OneHotEncoder(sparse = False)

#Entrenamos
ohe.fit(cat)

#Aplicamos 
cat_ohe = ohe.transform(cat)

#ponemos los nombres
cat_ohe = pd.DataFrame(cat_ohe, columns = ohe.get_feature_names_out(input_features = cat.columns)).reset_index(drop = True)

In [ ]:
cat_ohe

#### Data Frame Final


Seleccionamos las variables numericas para poder juntarlas a las cat_hoe

In [ ]:
num = df.select_dtypes('number').reset_index(drop= True)


Las juntamos todas en el dataframe final

In [ ]:
df_ml = pd.concat([cat_ohe, num], axis= 1)
df_ml

### Diseno de la modelizacion

#### Separacion predictores y targets


In [ ]:
x = df_ml.drop(columns= 'abandono')
y = df_ml['abandono']

#### Separacion train y test

In [ ]:
from sklearn.model_selection import train_test_split
train_x, test_x, train_y, test_y = train_test_split(x,y,test_size= 0.3)

### Entreamiento del modelo sobre train 

In [ ]:
from sklearn.tree import DecisionTreeClassifier
#Instanciar
ac = DecisionTreeClassifier(max_depth= 4)
#Entrenar
ac.fit(train_x,train_y)

### Prediccion y validacion sobre test

In [ ]:
#Prediccion
pred = ac.predict_proba(test_x)[:,1]
pred[:20]

In [ ]:
#Evaluacion
from sklearn.metrics import roc_auc_score

roc_auc_score(test_y,pred)

### Interpreatacion

#### Diagrama de arbol

In [ ]:
from sklearn.tree import plot_tree

plt.figure(figsize= (50,50))

plot_tree(ac,
          feature_names=test_x.columns,
          impurity= False,
          node_ids= True,
          proportion= True,
          rounded= True,
          precision= 2);

In [ ]:
pd.Series(ac.feature_importances_, index = test_x.columns).sort_values(ascending= False).plot(kind= 'bar',figsize= (30,20));

###  Explotacion

Incorporacion del scoring al dataframe principal.

In [ ]:
df['scoring_abandono'] = ac.predict_proba(df_ml.drop(columns = 'abandono'))[:,1]
df

Ejemplo de los 10 empleados con mayor probabilidad de dejar la empresa.

In [ ]:
df.sort_values(by= 'scoring_abandono', ascending= False)[0:10]

In [ ]:
df.boxplot(column= 'scoring_abandono', by ='puesto',figsize = (20,12))

### Guardar el resultado

In [ ]:
df

In [ ]:
# Save the DataFrame to an XLSX file
df.to_excel('Employee turnover costs calculation.xlsx')
import openpyxl
